# Neural Network

神经网络（Neural Network）是一种模拟生物神经系统的计算模型，广泛应用于机器学习和人工智能领域。神经网络由多个神经元（Neuron）组成，这些神经元通过连接（Connection）形成网络结构。以下是神经网络的基本概念和原理。

### 神经网络的基本概念

1. **神经元（Neuron）**：
   - 神经元是神经网络的基本单位，类似于生物神经元。
   - 每个神经元接收多个输入信号，通过加权求和和激活函数处理后，输出一个信号。

2. **权重（Weight）**：
   - 权重是连接神经元的参数，表示输入信号的重要性。
   - 权重在训练过程中不断调整，以最小化损失函数。

3. **偏置（Bias）**：
   - 偏置是一个额外的参数，用于调整神经元的输出。
   - 偏置在训练过程中也会不断调整。

4. **激活函数（Activation Function）**：
   - 激活函数用于引入非线性，使神经网络能够处理复杂的非线性问题。
   - 常见的激活函数包括 Sigmoid、ReLU（Rectified Linear Unit）、Tanh 等。

5. **层（Layer）**：
   - 神经网络由多个层组成，每层包含多个神经元。
   - 输入层（Input Layer）：接收输入数据。
   - 隐藏层（Hidden Layer）：处理输入数据，提取特征。
   - 输出层（Output Layer）：输出预测结果。

### 神经网络的工作原理

1. **前向传播（Forward Propagation）**：
   - 输入数据通过输入层传递到隐藏层，再通过隐藏层传递到输出层。
   - 每个神经元计算输入信号的加权和，经过激活函数处理后输出信号。
   - 输出层的输出即为神经网络的预测结果。

2. **损失函数（Loss Function）**：
   - 损失函数用于衡量神经网络的预测结果与真实值之间的差异。
   - 常见的损失函数包括均方误差（Mean Squared Error, MSE）、交叉熵损失（Cross-Entropy Loss）等。

3. **反向传播（Backward Propagation）**：
   - 反向传播用于计算损失函数相对于每个权重和偏置的梯度。
   - 通过链式法则，梯度从输出层逐层传递到输入层。

4. **梯度下降（[Gradient Descent](./Gradient_Descent.ipynb)）**：
   - 梯度下降用于更新权重和偏置，以最小化损失函数。
   - 常见的梯度下降算法包括批量梯度下降（Batch Gradient Descent）、随机梯度下降（Stochastic Gradient Descent, SGD）和小批量梯度下降（Mini-Batch Gradient Descent）。

### 神经网络的训练过程

1. **初始化**：
   - 初始化神经网络的权重和偏置。

2. **前向传播**：
   - 输入数据通过神经网络，计算输出结果。

3. **计算损失**：
   - 使用损失函数计算预测结果与真实值之间的差异。

4. **反向传播**：
   - 计算损失函数相对于每个权重和偏置的梯度。

5. **更新参数**：
   - 使用梯度下降算法更新权重和偏置。

6. **重复**：
   - 重复前向传播、计算损失、反向传播和更新参数的过程，直到损失函数收敛或达到预定的训练轮数。

## 代码示例: 用神经网络识别 MNIST 手写数字 的类别(标签, 0-9)

对于 [MNIST 数据集](./MNIST_dataset.ipynb#MNIST-dataset-shape) 的单个图像的形状是 [1,28,28],展平为向量后形状为 [784], 作为神经网络的输入.
$$
\begin{bmatrix}
x_0 \\
x_1  \\
x_2  \\
\vdots \\
x_{783}  \\
\end{bmatrix}
$$
输入层: 784个输入神经元, 隐藏层: 建议选择:一层,256个神经元, 输出层: 10个神经元, 分别对应0-9的数字.

In [ ]:
# 1. 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 2. 定义数据预处理和加载数据
# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为张量
    transforms.Normalize((0.5,), (0.5,))  # 归一化到 [-1, 1]
])

# 加载训练集和测试集
train_dataset = datasets.MNIST(root='../data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='../data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# 3. 定义神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# 超参数
input_size = 784  # 输入层大小（28x28 的图像展平为向量）
hidden_size = 256  # 隐藏层大小
output_size = 10  # 输出层大小（10 个类别）
learning_rate = 0.001
epochs = 10
# 4. 初始化模型、损失函数和优化器
model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# 5. 训练神经网络
for epoch in range(epochs):
    # 内层循环使用 train_loader 加载数据, 进行小批量的数据读取
    for i, (images, labels) in enumerate(train_loader):
        # if i == 3: # 为了快速验证代码的正确性，只迭代3次
        #     break
        # 内层每次迭代时，都会进行一次 梯度下降算法, 包括5个步骤
        # 将图像展平为向量
        # print(f"batch_idx: {i}, images.shape: {images.shape}, labels.shape: {labels.shape}")
        # print(f"labels: {labels}")
        images = images.view(-1, 28*28) # .shape = (64, 784)
        # print(images.shape) 
        
        outputs = model(images) # 1.前向传播: 计算输出
        loss = criterion(outputs, labels) # 2.计算输出和标签之间的损失

        # 这三步的顺序
        optimizer.zero_grad() # 3.梯度清零: 以确保每次计算梯度时不会累加上一次计算的梯度
        loss.backward() # 4.反向传播: 计算梯度
        optimizer.step() # 5.更新参数

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
# 保存模型
simple_nn_model_dir = '../output/weights/simple_nn_model/'
torch.save(model.state_dict(), simple_nn_model_dir + 'simple_nn_model.pth')
# 6. 评估神经网络
# 评估模型在测试集上的性能
model.eval()  # 设置模型为评估模式
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1792x28 and 784x256)